# Tratamento inicial de dados

Aqui iniciaremos nosso tratemento inicial de dados, vamos converter o JSON com as matérias coletadas para um Dataframe e incluir as colunas com os resumos de cada matéria para cada persona.

Estrutura do dataframe:
- id: Identificador associado a cada matéria
- categoria: Categoria a qual a matéria pertence
- titulo: Titulo da matéria
- subtitulo: Subtítulo da matéria
- texto: Texto completo da matéria
- resumo_persona_1: Resumo gerado usando a LLM escolhida para a persona 1
- resumo_persona_2: Resumo gerado usando a LLM escolhida para a persona 2
- resumo_persona_3: Resumo gerado usando a LLM escolhida para a persona 3

### Importando bibliotecas utilizadas

In [1]:
import json
import pandas as pd
import warnings
from summary_builder import SummaryBuilder

warnings.filterwarnings('ignore')


### Leitura dos dados

In [29]:
with open("./materias.json", "r") as f:
    materias = json.load(f)

with open("./personas.json", "r") as f:
    personas = json.load(f)

# print(json.dumps(materias, indent=2))
# print(json.dumps(personas, indent=2))

In [30]:
df = pd.DataFrame(materias["materias"], columns=["id", "categoria", "url", "titulo", "subtitulo", "texto", "resumo_persona_1", "resumo_persona_2", "resumo_persona_3"])

df.set_index("id", inplace=True)

df["resumo_persona_1"].fillna("", inplace=True)
df["resumo_persona_2"].fillna("", inplace=True)
df["resumo_persona_3"].fillna("", inplace=True)

df

,categoria,url,titulo,subtitulo,texto,resumo_persona_1,resumo_persona_2,resumo_persona_3
id,,,,,,,,
4,saude,https://g1.globo.com/saude/bem-estar/noticia/2...,Só um país é autossuficiente para dieta saudáv...,Pesquisa mapeou o quanto a produção de sete gr...,Em uma era de crescente nacionalismos e disrup...,,,


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11 entries, 1 to 11
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   categoria         11 non-null     object
 1   url               11 non-null     object
 2   titulo            11 non-null     object
 3   subtitulo         11 non-null     object
 4   texto             11 non-null     object
 5   resumo_persona_1  11 non-null     object
 6   resumo_persona_2  11 non-null     object
 7   resumo_persona_3  11 non-null     object
dtypes: object(8)
memory usage: 792.0+ bytes


### Gerando texto para cada persona

In [ ]:
summary_builder = SummaryBuilder()
resumos = {
    "persona_1": [],
    "persona_2": [],
    "persona_3": []
}

for index, row in df.iterrows():
    print(f"Gerando resumos do texto {index} - Título: {row["titulo"]}")
    texto = row["texto"]
    texto_categoria = row["categoria"]

    for persona in personas["personas"]:
        # if persona["id"] != 3:
        #     continue
        print(f"Gerando texto para: {persona['nome']}")

        _id = persona["id"]
        preferencias = persona["scores"]
        time_do_coracao = persona["time_do_coracao"]
        nome_persona = persona['nome']
        
        resumo = summary_builder.get(nome_persona, texto, texto_categoria, preferencias, time_do_coracao)
        resumos[f"persona_{_id}"].append(resumo)
        
    
    print("\n")
        

Gerando resumos do texto 4 - Título: Só um país é autossuficiente para dieta saudável, diz estudo
Gerando texto para: renata
Elabore um resumo para Renata sobre a notícia com tema principal em saúde. Como o interesse dela por saúde é moderado, mantenha o resumo breve. Foque também nos temas de economia e política, pois ela tem alto interesse neles.
 Uma pesquisa recente na revista Nature Food analisou a capacidade de 186 países de alimentar suas populações sem comércio internacional, focando na dieta Livewell, que promove uma alimentação saudável e sustentável. Apenas a Guiana alcançou autossuficiência completa nos sete grupos de alimentos essenciais. O Brasil consegue produzir cinco desses grupos, mas enfrenta déficits em verduras e peixes. O estudo revela disparidades globais, com países europeus superproduzindo carne e laticínios, enquanto nações africanas e algumas da América Latina têm dificuldades em diversos grupos alimentares. O pesquisador Jonas Stehl alerta que, embora a auto

### Salvando resumos gerados no Dataframe

In [ ]:
for i in range(len(personas["personas"])):
    # if (i+1) != 1:
    #     continue
    df[f"resumo_persona_{i + 1}"] = resumos[f"persona_{i + 1}"]
    

In [ ]:
df

In [ ]:
df.loc[7, "resumo_persona_1"]

### Exportando para .csv

In [ ]:
df.to_csv("resumos.csv", sep=";", encoding="utf-8")

In [ ]:
df.to_json("resumos.json")